---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [29]:
import pandas as pd
import numpy as np

def load_dataset():
    #loadind dataset
    train_df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_df = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    return train_df, test_df

In [30]:
train_df, test_df = load_dataset()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


## Check datasets

In [31]:
def train_test(train, test):
    # Since there are some columns only in training set, they should be removed 
    # so that the information in training and test datasets are aligned
    # pandas ver on Coursera is still 0.16.1
    if list(train.columns) == list(test.columns): return 
    else:
    #    drop_columns = list(train.columns)
    #    for column in list(test.columns):
    #        drop_columns.remove(column)
    #    print(drop_columns)
    #    train.drop(columns = drop_columns, inplace = True)
        X_train = train.loc[:, list(test.columns)]
        Y_train = train.loc[:, 'compliance']
        X_test = test_df
        return X_train, Y_train, X_test

In [32]:
X_train, Y_train, X_test = train_test(train_df, test_df)
#Y_train is a Series
del(train_df)
del(test_df)

In [33]:
X_train.iloc[:, 9:].head()

,city,state,zip_code,non_us_str_code,country,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,CHICAGO,IL,60606,NaN,USA,2004-03-16 11:40:00,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN
1,Detroit,MI,48208,NaN,USA,2004-04-23 12:30:00,2005-05-06 13:30:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN
2,DETROIT,MI,48223,NaN,USA,2004-04-26 13:40:00,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,DETROIT,MI,48214,NaN,USA,2004-04-26 13:30:00,NaN,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,DETROIT,MI,48206,NaN,USA,2004-04-26 13:00:00,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [34]:
print(len(X_train['violation_code'].unique()))
print(len(X_train['zip_code'].unique()))
print(len(X_train['country'].unique()))
print(len(X_train['state'].unique()))
print(len(X_train['city'].unique()))

235
5643
5
60
5184


In [35]:
def count_null(dataset):
    #count how many NaN values in each column
    return dataset.isnull().sum(axis = 0)

In [36]:
print(count_null(X_train), len(X_train))
print(count_null(X_test), len(X_test))
print(count_null(Y_train), len(Y_train[Y_train.values == 0]), len(Y_train[Y_train.values == 1]),
      len(Y_train))

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                     34
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           4
city                               0
state                             93
zip_code                           1
non_us_str_code               250303
country                            0
ticket_issued_date                 0
hearing_date                   12491
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
grafitti_status               250305
d

### Note: All tickets where the violators were found not responsible are not considered during evaluation.

In [37]:
def drop_nan_label(X_train, Y_train):
    #remove NaN labels and their features
    X_train['label'] = Y_train
    Y_train.dropna(inplace = True)
    X_train = X_train.iloc[Y_train.index, :-1]
    return X_train, Y_train

In [38]:
X_train, Y_train = drop_nan_label(X_train, Y_train)

#### Now try addresses.csv & latlons.csv to see if there is any information we can apply to the training and test datasets.

In [39]:
addresses = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')
addresses.head(), latlons.head()

(   ticket_id                      address
 0      22056       2900 tyler, Detroit MI
 1      27586     4311 central, Detroit MI
 2      22062  1449 longfellow, Detroit MI
 3      22084  1441 longfellow, Detroit MI
 4      22093   2449 churchill, Detroit MI,
                                   address        lat        lon
 0  4300 rosa parks blvd, Detroit MI 48208  42.346169 -83.079962
 1                14512 sussex, Detroit MI  42.394657 -83.194265
 2                3456 garland, Detroit MI  42.373779 -82.986228
 3                5787 wayburn, Detroit MI  42.403342 -82.957805
 4              5766 haverhill, Detroit MI  42.407255 -82.946295)

In [40]:
# Join latlons info into addresses
addresses = addresses.merge(latlons, how = 'left', on = 'address')
addresses.head()

,ticket_id,address,lat,lon
0,22056,"2900 tyler, Detroit MI",42.390729,-83.124268
1,27586,"4311 central, Detroit MI",42.326937,-83.135118
2,22062,"1449 longfellow, Detroit MI",42.380516,-83.096069
3,22084,"1441 longfellow, Detroit MI",42.380570,-83.095919
4,22093,"2449 churchill, Detroit MI",42.145257,-83.208233


In [41]:
addresses.isnull().sum(axis = 0)

ticket_id    0
address      0
lat          8
lon          8
dtype: int64

Seems to be okay, let's merge latlons and address info into the training and test datasets.
Also, data of columns 'violation_zip_code', 'grafitti_status', and 'non_us_str_code' are most NaN values. I'll drop them as well.

In [42]:
def update_dataset(X_train, X_test, addresses):
    X_train = X_train.merge(addresses, on = 'ticket_id', how = 'left')
    X_test = X_test.merge(addresses, on = 'ticket_id', how = 'left')
    columns = list(X_train.columns)
    columns.remove('violation_zip_code')
    columns.remove('grafitti_status')
    columns.remove('non_us_str_code')
    X_train = X_train.loc[:, columns]
    X_test = X_test.loc[:, columns]
    return X_train, X_test

In [43]:
X_train, X_test = update_dataset(X_train, X_test, addresses)
X_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,address,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,3.0,S. WICKER,CHICAGO,IL,...,250.0,20.0,10.0,25.0,0.0,0.0,305.0,"2900 tyler, Detroit MI",42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,2959.0,Martin Luther King,Detroit,MI,...,750.0,20.0,10.0,75.0,0.0,0.0,855.0,"4311 central, Detroit MI",42.326937,-83.135118
2,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,2755.0,E. 17TH,LOG BEACH,CA,...,250.0,20.0,10.0,25.0,0.0,0.0,305.0,"6478 northfield, Detroit MI",42.145257,-83.208233
3,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,476.0,Garfield,Clinton,MI,...,750.0,20.0,10.0,75.0,0.0,0.0,855.0,"8027 brentwood, Detroit MI",42.433466,-83.023493
4,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,8228.0,Mt. Elliott,Detroit,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,"8228 mt elliott, Detroit MI",42.388641,-83.037858


In [44]:
print(count_null(X_train), len(X_train))
print(count_null(X_test), len(X_test))
print(X_train.dtypes)

ticket_id                        0
agency_name                      0
inspector_name                   0
violator_name                   26
violation_street_number          0
violation_street_name            0
mailing_address_str_number    2558
mailing_address_str_name         3
city                             0
state                           84
zip_code                         1
country                          0
ticket_issued_date               0
hearing_date                   227
violation_code                   0
violation_description            0
disposition                      0
fine_amount                      0
admin_fee                        0
state_fee                        0
late_fee                         0
discount_amount                  0
clean_up_cost                    0
judgment_amount                  0
address                          0
lat                              2
lon                              2
dtype: int64 159880
ticket_id                        0


In [45]:
print(len(X_train['address'].unique()))

71901


I think ticket id might lead to data leakage and it would not provide any useful information whether the Blight ticket would be paid or not, so I would like to remove it when training. Also I would like to check if the data of int and float types are highly correlated to the label.

Since many data in the datasets are strings, I would like to check the correlations and then decide whether to manipulate the string data. 

In [46]:
def print_corr():
    print('corr with fine amount: ', Y_train.corr(X_train['fine_amount']))
    print('corr with admin fee: ', Y_train.corr(X_train['admin_fee']))
    print('corr with state fee: ', Y_train.corr(X_train['state_fee']))
    print('corr with late fee: ', Y_train.corr(X_train['late_fee']))
    print('corr with discount amount: ', Y_train.corr(X_train['discount_amount']))
    print('corr with clean up cost: ', Y_train.corr(X_train['clean_up_cost']))
    print('corr with judgment amount: ', Y_train.corr(X_train['judgment_amount']))

print_corr()

corr with fine amount:  -0.00566661237894
corr with admin fee:  nan
corr with state fee:  nan
corr with late fee:  -0.00564318791216
corr with discount amount:  -0.0038991014002
corr with clean up cost:  nan
corr with judgment amount:  -0.00566944000286


In [47]:
print(len(X_train['violation_code'].unique()))
print(len(X_train['zip_code'].unique()))
print(len(X_train['country'].unique()))
print(len(X_train['state'].unique()))
print(len(X_train['city'].unique()))
print(len(X_train['disposition'].unique()))

189
4623
5
60
4093
4


In [48]:
X_train.join(pd.get_dummies(X_train['country'], prefix = 'country', drop_first = True))

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,mailing_address_str_number,mailing_address_str_name,city,state,...,discount_amount,clean_up_cost,judgment_amount,address,lat,lon,country_Cana,country_Egyp,country_Germ,country_USA
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,3.0,S. WICKER,CHICAGO,IL,...,0.0,0.0,305.0,"2900 tyler, Detroit MI",42.390729,-83.124268,0,0,0,1
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,2959.0,Martin Luther King,Detroit,MI,...,0.0,0.0,855.0,"4311 central, Detroit MI",42.326937,-83.135118,0,0,0,1
2,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,2755.0,E. 17TH,LOG BEACH,CA,...,0.0,0.0,305.0,"6478 northfield, Detroit MI",42.145257,-83.208233,0,0,0,1
3,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,476.0,Garfield,Clinton,MI,...,0.0,0.0,855.0,"8027 brentwood, Detroit MI",42.433466,-83.023493,0,0,0,1
4,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,8228.0,Mt. Elliott,Detroit,MI,...,0.0,0.0,140.0,"8228 mt elliott, Detroit MI",42.388641,-83.037858,0,0,0,1
5,18733,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,8228.0,Mt. Elliott,Detroit,MI,...,0.0,0.0,140.0,"8228 mt elliott, Detroit MI",42.388641,-83.037858,0,0,0,1
6,28204,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Inc, Nanno",15307.0,SEVEN MILE,1537.0,E. Seven Mile,Detroit,MI,...,0.0,0.0,855.0,"15307 seven mile, Detroit MI",42.435773,-82.963348,0,0,0,1
7,18743,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Gardner Resale, GAR",9100.0,VAN DYKE,91.0,Van Dyke,Detroit,MI,...,0.0,0.0,855.0,"9100 van dyke, Detroit MI",42.395765,-83.022333,0,0,0,1
8,18741,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Hardaway, Kevin",20024.0,SCHAEFER,224.0,Schaefer,Detroit,MI,...,0.0,0.0,855.0,"20024 schaefer, Detroit MI",42.440190,-83.180488,0,0,0,1
9,18978,"Buildings, Safety Engineering & Env Department","Williams, Darrin","TLC Hand Car Wash, a/k/a",9425.0,VAN DYKE,9425.0,Van Dyke,Detroit,MI,...,0.0,0.0,855.0,"9425 van dyke, Detroit MI",42.399222,-83.023161,0,0,0,1


In [51]:
X_train['state_IL'] = X_train['state'] == 'IL'
X_train['state_IL']

0          True
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
159850    False
159851    False
159852    False
159853    False
159854    False
159855    False
159856    False
159857    False
159858    False
159859    False
159860    False
159861    False
159862    False
159863    False
159864    False
159865    False
159866    False
159867    False
159868    False
159869    False
159870    False
159871    False
159872    False
159873    False
159874    False
159875    False
159876    False
159877    False
159878    False
159879    False
Name: state_IL, dtype: b

In [8]:
#import necessary ML modules
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

In [60]:
import pandas as pd
import numpy as np

#import necessary ML modules
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def blight_model():
    #read dataset
    train_df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_df = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    #remove NaN values in compliance label
    train_df = train_df.loc[train_df['compliance'].isnull() == 0, :]
    
    #merge address and latlons data
    addresses = addresses.merge(latlons, how = 'left', on = 'address')
    del(latlons)
    
    #update address and latlons data onto training and test datasets
    train_df = train_df.merge(addresses, on = 'ticket_id', how = 'left')
    test_df = test_df.merge(addresses, on = 'ticket_id', how = 'left')
    del(addresses)
    
    #define Y_train
    Y_train = train_df['compliance']
    
    #remove columns with most NaN values in training and test datasets
    columns = list(test_df.columns)
    columns.remove('violation_zip_code')
    columns.remove('grafitti_status')
    columns.remove('non_us_str_code')
    
    #remove columns only in training set and define X_train and X_test
    X_train = train_df.loc[:, columns]
    X_test = test_df.loc[:, columns]
    
    #change string dtype to categorical, also lat & lon
    X_train.loc[X_train['lat'].isnull() == 0,'lat'] = X_train['lat'] // (0.2)
    X_train.loc[X_train['lat'].isnull() == 1,'lat'] = 'NA'
    X_train.loc[X_train['lon'].isnull() == 0,'lon'] = X_train['lon'] // (0.2)
    X_train.loc[X_train['lon'].isnull() == 1,'lon'] = 'NA'
    X_test.loc[X_test['lat'].isnull() == 0,'lat'] = X_test['lat'] // (0.2)
    X_test.loc[X_test['lat'].isnull() == 1,'lat'] = 'NA'
    X_test.loc[X_test['lon'].isnull() == 0,'lon'] = X_test['lon'] // (0.2)
    X_test.loc[X_test['lon'].isnull() == 1,'lon'] = 'NA'
    X_train['lat'].astype('category')
    X_train['lon'].astype('category')
    X_test['lat'].astype('category')
    X_test['lon'].astype('category')
    
    '''
    used data columns:
    ticket_id                       int64
    disposition                    object
    state                          object
    lon                           float64
    country                        object
    lat                           float64
    judgment_amount               float64
    '''
    # Convert NaN to "NA" in columns to convert to type category
    convert_columns={'country': 'category',
                     'state': 'category',
                     'lon': 'category',
                     'disposition': 'category',
                     'lat': 'category'
                    }
    
    for col, col_type in convert_columns.items():
        if col_type == 'category':
            X_train[col] = X_train[col].replace(np.nan, "NA", regex=True).astype(col_type)
            X_test[col] = X_test[col].replace(np.nan, "NA", regex=True).astype(col_type)
        elif col_type == 'int':
            X_train[col] = X_train[col].replace(np.nan, 0, regex=True).astype(col_type)
            X_test[col] = X_test[col].replace(np.nan, 0, regex=True).astype(col_type)

    # Remove unneeded columns from X sets  
    # street_num, street_name, mailing_address_str_name, address, violation_zip_code are useless info
    # all kind of disclosed with lat and lon info
    # date info are also useless
    # violation_description is the same thing as violation_code
    # judgment_amount is the sumup of all other fines and fees, so it should be okay to only include judgment_amount
    # finally, agency name might not be related with the compliance
    # len of unique violator_name: 119993, too many to have valuable info for prediction
    # lat and lon are kind of showing the same info as city/state/country
    # city and zip_code have more than 4000 types, making the IDE crash, so I'll take lat and lon as geo info
    common_cols_to_drop = ['mailing_address_str_number', 'violation_street_number', 'violation_street_name',
                           'mailing_address_str_name', 'address', 'admin_fee', 'state_fee', 'late_fee',
                           'ticket_issued_date', 'hearing_date', 'violation_description', 'violation_code',
                           'inspector_name', 'fine_amount', 'discount_amount', 'clean_up_cost',
                           'agency_name', 'violator_name', 'zip_code', 'city']
    for drop in common_cols_to_drop:
        columns.remove(drop)
    #print(X_train['lat'].unique(), X_train['lon'].unique())
    X_train = X_train.loc[:, columns].set_index('ticket_id')
    X_test = X_test.loc[:, columns].set_index('ticket_id')
    
    #print(len(X_train.columns), len(X_test.columns))    
    # Convert category columns to integers
    cat_columns = list(X_train.select_dtypes(['category']).columns)
    
    #Should apply get_dummies to do one_hot_coding......
    for column in cat_columns:
        one_hot_keys = list(X_train[column].unique())
        for key in one_hot_keys:
            X_train['%s_%s' %(column, key)] = X_train[column] == key
            X_test['%s_%s' %(column, key)] = X_test[column] == key
            #X_test = X_test.join(pd.get_dummies(X_test[column], prefix = column, drop_first = True))
    X_train = X_train.iloc[:, [3] + list(range(6, len(X_train.columns)))]
    X_test = X_test.iloc[:, [3] + list(range(6, len(X_test.columns)))]
    
    
    #final check before start learning
    #return X_train #OK
    #print(X_train.isnull().sum(), X_test.isnull().sum()) #OK
    #print(len(X_train.columns), len(X_test.columns)) #OK
    # Start learning    
    
    scalar = MinMaxScaler()
    X_train['judgment_amount'] = scalar.fit_transform(X_train['judgment_amount'])
    X_test['judgment_amount'] = scalar.transform(X_test['judgment_amount'])
    
    #print(X_train.isnull().sum(), X_test.isnull().sum())
    clf = GradientBoostingClassifier(learning_rate = 0.3, max_depth = 3,random_state = 0)
    
    #grid_values = {'learning_rate': [0.01, 0.03, 0.1, 0.3, 1.], 'max_depth': [3,4,5]}
    """(GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.3, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_split=1e-07, min_samples_leaf=1,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               n_estimators=100, presort='auto', random_state=0,
               subsample=1.0, verbose=0, warm_start=False),
         0.79182409580805868)"""
    #grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
    #grid_clf_auc = GridSearchCV(clf, param_grid = best_param, scoring = 'roc_auc')
    #grid_clf_auc.fit(X_train, Y_train)
    #return grid_clf_auc.best_estimator_, grid_clf_auc.best_score_
    clf.fit(X_train, Y_train)
    probs = clf.predict_proba(X_test)[:, 1]
    result = pd.Series(probs, index=X_test.index)
    
    return result


In [61]:
result = blight_model()
result

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/

ticket_id
284932    0.044845
285362    0.010644
285361    0.056418
285338    0.052363
285346    0.061762
285345    0.052363
285347    0.056297
285342    0.787703
285530    0.009632
284989    0.031257
285344    0.056297
285343    0.013756
285340    0.013756
285341    0.056297
285349    0.057580
285348    0.044395
284991    0.031445
285532    0.020864
285406    0.020864
285001    0.020864
285006    0.013756
285405    0.016028
285337    0.009854
285496    0.056297
285497    0.052363
285378    0.016028
285589    0.027861
285585    0.045461
285501    0.061762
285581    0.010644
            ...   
376367    0.020864
376366    0.033130
376362    0.205889
376363    0.228079
376365    0.020864
376364    0.033130
376228    0.032840
376265    0.025521
376286    0.322266
376320    0.032840
376314    0.024428
376327    0.320287
376385    0.314270
376435    0.196555
376370    0.867186
376434    0.056782
376459    0.061762
376478    0.007272
376473    0.032840
376484    0.021868
376482    0.056448
37